## Lab 9: Starter Code

### Part A: Using Statsmodels

In [3]:
import pandas as pd
import numpy as np
# Statsmodels logistic regression is sm.Logit
import statsmodels.api as sm

/Users/665066/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
# Read in the data collegeadmissions.csv as df
df =  

SyntaxError: invalid syntax (<ipython-input-4-ba56cbf64eb1>, line 2)

In [ ]:
# show first few rows


In [ ]:
# create dummies from rank and join it to df
df = df.join(pd.get_dummies(df['rank']))

In [ ]:
# show first rows again


In [ ]:
X = df[['gre', 'gpa', 1, 2, 3,]]
X = sm.add_constant(X)
y = df['admit']

lm = sm.Logit(y, X)
result = lm.fit()

result.summary()

In [ ]:
print df.admit.mean()

In [ ]:
# You can easily convert these into odds using numpy.exp()
print np.exp(result.params)

The above makes it more clear that a schools rank as it approaches 4 decreases the odds of getting admitted.
The accuracy of the model with all features (removing one rank) is ~70%.

In [ ]:
predicted = result.predict(X)
threshold = 0.5
predicted_classes = (predicted > threshold).astype(int)
from sklearn.metrics import accuracy_score
accuracy_score(y, predicted_classes)

### Part B: ROC/AUC

Below is some code to walk through confusion matrices. It'll be useful for working through the Titanic problem.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

Below the ROC curve is based on various thresholds: it shows with a false positive rate (x-axis) ~0, it also expects a true positive rate (y-axis) ~0 (the same, ish, for the top right hand of the figure).

The second chart, which does not play with thesholds, shows the one true TPR and FPR point, joined to 0,0 and 1,1.

The first chart will be more effective as you compare models and determine where the decision line should exist for the data. The second simplifies the first in case this idea of thresholds is confusing.

In [ ]:
fpr = roc_curve(df[['admit']], predicted)[0]
tpr = roc_curve(df[['admit']], predicted)[1]

In [ ]:
plt.plot(fpr, tpr)

Finally, you can use the `roc_auc_score` function to calculate the area under these curves (AUC).

In [ ]:
roc_auc_score(df['admit'], predicted_classes)

### Part C: Using SKLEARN

#### Note: sklearn also has logistic regression:
```
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(X, y)
```

### Titanic Problem

In [ ]:
# import survival.csv as titanic


In [ ]:
# show first few rows


In [ ]:
# set 'PassengerId' as the index with inplace=True


In [ ]:
# Show histogram of age split by "survived" and "not survived"
%matplotlib inline


In [ ]:
# show last few rows of data


In [ ]:
# summary of data


In [ ]:
from sklearn import grid_search, cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn import metrics 

# select your columns for inputs and create X_train, X_test, y_train, y_test
feature_set = titanic[['is_male', "Location", "Class", 'Fare', 'Age', 'had_parents', 'had_siblings']]

# Create grid_search.GridSearchCV and tune "C" and "class_weight"
# "C": Inverse of regularization strength; must be a positive float. 
# Smaller C values specify stronger regularization.

# "class_weight" gives automatically more weight to "rare" class data.
# Use these values: 'C': [10**-i for i in range(-5, 5)], 'class_weight': [None, 'balanced']
gs = grid_search.GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=,
    cv=cross_validation.KFold(),
    scoring='roc_auc'
)

gs.fit(X_train, y_train)
gs.grid_scores_
#print gs.best_estimator_

In [ ]:
print gs.best_estimator_

In [ ]:
y_pred = gs.best_estimator_.predict(X_test)

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix

In [ ]:
# Show confusion matrix in a separate window
plt.matshow(confusion_matrix)
plt.title('Confusion matrix')
plt.colorbar()
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
y_pro = gs.best_estimator_.predict_proba(X_test)
fpr = roc_curve(y_test, y_pred)[0]
tpr = roc_curve(y_test, y_pred)[1]
plt.plot(fpr, tpr)

In [ ]:
roc_auc_score(y_test, y_pred)

In [ ]:
y_pro = gs.best_estimator_.predict_proba(X_test)
fpr = roc_curve(y_test, y_pro[:,1])[0]
tpr = roc_curve(y_test, y_pro[:,1])[1]
plt.plot(fpr, tpr)